<font color = blue> First of all, we have to import some libraries: </font>


In [1]:
import numpy as np
import pandas as pd
# Load the H2O library and start up the H2O cluter locally on your machine
import h2o
from h2o.automl import H2OAutoML

# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 days 1 hour 52 mins
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_emachado_449c9r
H2O cluster total nodes:,1
H2O cluster free memory:,6.231 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


<font color = blue> Data loading and merging to get the 'logerror' which is our target: </font>

In [2]:
prop = pd.read_csv('./data/properties_2016.csv')
train = pd.read_csv('./data/train_2016_v2.csv')
data = pd.merge(train, prop, on='parcelid', how='left')
data.head()


/Users/emachado/miniconda3/envs/ironhack/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [3]:
data.to_csv('data.csv')


In [4]:
data = 'data.csv'

In [5]:
data = h2o.import_file(data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
data.describe()

Rows:90275
Cols:61




,C1,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
type,int,int,real,time,int,int,int,real,int,int,int,real,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,int,int,int,real,int,int,int,int,int,int,int,enum,real,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,int,int,int,int,real,enum,int,int
mins,0.0,10711738.0,-4.605,1451606400000.0,1.0,2.0,100.0,0.0,0.0,4.0,1.0,1.0,66.0,44.0,2.0,2.0,1056.0,560.0,44.0,257.0,6037.0,1.0,1.0,0.0,0.0,,1.0,33339295.0,-119447865.0,167.0,1.0,28.0,1.0,1.0,1.0,0.0,31.0,,60371011.101001,3491.0,1286.0,6952.0,95982.0,0.0,7.0,1.0,4.0,1.0,25.0,18.0,1885.0,1.0,,100.0,22.0,2015.0,22.0,49.08,,6.0,60371011101001.0
mean,45137.0,12984656.108712263,0.011457219606757132,1465669839260.0396,1.8163719120252944,7.2298850574712645,713.5813953488372,2.2794738299639996,3.0318692882857934,4.0,5.565406875392233,2.309216212272568,66.0,1347.9740373395566,1773.1859865646002,1745.4545305855427,1404.5454545454545,2380.0900673400674,1355.5466744457412,2302.546318289786,6048.870551093877,1.1878838347038618,2.241231073148283,1.8120052109429803,345.5212279119484,,3.9269793152639094,34005410.693691485,-118198868.30685121,29110.164176624032,1.0,519.8276573787409,1.0,1.0,1.0,202.69972340282004,261.83217945167564,,60491794.811503045,33761.33285107155,2525.4207698698433,190646.54023658775,96586.13118351072,1.4785156466352811,7.0,1.008743442418187,6.010033444816053,1.1104142032114879,310.1421012849584,311.69473684210527,1968.5328701169578,1.4407389402041815,,180093.3937482616,457672.6273567138,2015.0,278335.3250216006,5983.9759267301115,,13.402692091979809,60491508546032.67
maxs,90274.0,162960842.0,4.737,1483056000000.0,13.0,21.0,1555.0,20.0,16.0,4.0,12.0,20.0,66.0,7625.0,22741.0,20013.0,1584.0,22741.0,8352.0,7224.0,6111.0,5.0,20.0,24.0,7339.0,,24.0,34816009.0,-117554924.0,6971010.0,1.0,1750.0,1.0,1.0,1.0,8800.0,275.0,,61110091.00101999,396556.0,3101.0,764167.0,399675.0,18.0,7.0,4.0,13.0,143.0,2678.0,1366.0,2015.0,4.0,,9948100.0,27750000.0,2015.0,24500000.0,321936.09,,99.0,61110091001020.0
sigma,26060.292112714316,2504510.48830177,0.1610788353671866,7410679638.770435,2.974167945353214,2.7161956543833963,437.4341981839826,1.0042709877071896,1.1564355198675522,0.0,1.9006016285969713,0.9761724992343213,0.0,652.3990255999324,928.162392720824,909.9411657489277,110.10821123702891,1068.2071815335598,673.7034879321604,1346.2552421499329,20.663460805186304,0.48417274152432316,0.9631416217273224,0.6087605728316425,267.01591805179055,,3.6843820505313793,264965.37555690145,360603.1997035552,121721.30865440048,0.0,155.0542101242478,0.0,0.0,0.0,307.47152682954294,5.182900523458313,,205054.8672682629,46672.3938629122,805.6948419923107,166228.91057181926,3661.339093751154,2.8196270435222055,0.0,0.10083013967788877,0.4372346737754409,0.7972352624935549,216.7218688952167,346.3548504335012,23.76347471454321,0.5444981475501466,,209129.88816860018,554884.3989425417,0.0,400495.46489261574,6838.87695629231,,2.7159655039015123,204660503527.1381
zeros,1,0,847,0,0,0,0,1165,1421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,290,9209,,0,0,0,0,0

In [7]:
data.shape

(90275, 61)

In [8]:
y = 'logerror'

In [9]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [10]:
print(train.nrow)
print(valid.nrow)
print(test.nrow)

63207
13496
13572


In [20]:
aml1 = H2OAutoML(seed = 1, project_name = "zillow_prize")
aml1.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [21]:
aml1.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20190803_164221,0.0273481,0.165373,0.0273481,0.0684925,nan
StackedEnsemble_BestOfFamily_AutoML_20190803_164221,0.0273787,0.165465,0.0273787,0.0684103,nan
XRT_1_AutoML_20190803_164221,0.0274068,0.16555,0.0274068,0.0697122,nan
XRT_1_AutoML_20190806_233448,0.0274068,0.16555,0.0274068,0.0697122,nan
XRT_1_AutoML_20190806_231931,0.0274092,0.165557,0.0274092,0.0697447,nan
GBM_grid_1_AutoML_20190806_231931_model_5,0.0274462,0.165669,0.0274462,0.0686457,nan
GBM_grid_1_AutoML_20190806_233448_model_5,0.0274462,0.165669,0.0274462,0.0686457,nan
XGBoost_1_AutoML_20190806_155834,0.0274477,0.165673,0.0274477,0.068915,nan
XGBoost_1_AutoML_20190806_233448,0.0274477,0.165673,0.0274477,0.068915,nan
XGBoost_1_AutoML_20190806_231931,0.0274477,0.165673,0.0274477,0.068915,nan


In [22]:
aml2 = H2OAutoML(seed = 1, project_name = "zillow_prize_2")
aml2.train(y = y, training_frame = data)

AutoML progress: |███ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_b6fe665b4cf7ff0ef3ffa71baa294925> was cancelled by the user.

In [ ]:
aml2.leaderboard.head()

In [ ]:
pred1 = aml1.predict(test)
pred1.head()

In [ ]:
perf1 = aml1.leader.model_performance(test)
perf1

In [ ]:
pred2 = aml2.predict(test)
pred2.head()

In [ ]:
perf2 = aml2.leader.model_performance(test)
perf2